In [6]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

In [7]:
df_train = pd.read_csv('../../../datasets/parte2/treino/dataset_prepared.csv', na_filter= False)
df_test = pd.read_csv('../../../datasets/parte2/teste/dataset_prepared.csv')

In [8]:
X = df_train.drop('injection', axis=1)
y = df_train['injection']

In [9]:
param_grid = {
    'criterion': ['gini', 'entropy','ratio'],
    'max_depth': [5,7,9,11,13,15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}
model = RandomForestClassifier(n_estimators=100, random_state=2023, n_jobs=-1)

# Use GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, refit=True, verbose=2,cv=10)
grid_search.fit(X, y)
model = grid_search.best_estimator_

Fitting 10 folds for each of 162 candidates, totalling 1620 fits
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.4s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_leaf=1, min_samples_split=2; total time=   0.3s
[CV] END criterion=gini, max_depth=5, min_samples_lea

/home/goncalo/miniconda3/envs/daa310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
540 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "/home/goncalo/miniconda3/envs/daa310/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/goncalo/miniconda3/envs/daa310/lib/python3.10/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/home/goncalo/miniconda3/envs/daa310/lib/python3.10/site-packages/sklearn/base.py", line 637, in _validat

In [10]:
predictions = model.predict(df_test)

df_predicoes = pd.DataFrame({'Result': predictions})
df_predicoes['RowId'] = range(1, len(predictions) + 1)
df_predicoes = df_predicoes[['RowId', 'Result']]

df_predicoes.to_csv('../../../datasets/parte2/teste/previsoesRandom.csv', index=False)